# Building a Catalog

In [ ]:
from classes.source_catalog import GetCnilCatalog

url = 'https://www.data.gouv.fr/api/1/organizations/534fff61a3a7292c64a77d59/catalog'
headers = {'accept': 'application/json'}
url_add = 'https://www.data.gouv.fr/fr/organizations/cnil/datasets.csv'
instance1 = GetCnilCatalog(url, headers, url_add)
data = instance1.fetch_data_from_api()
data = data['@graph']
table_name = 'title'
download_url = 'downloadURL'
table_id = 'identifier' 
file_format= 'format'
last_update= 'modified' 
accessURL = '@id'
df_catalog = instance1.response_to_dataframe(data=data, table_name=table_name, download_url=download_url, table_id=table_id, file_format=file_format, last_update=last_update, accessURL=accessURL)
df_dataset = instance1.load_additional_info()
df_catalog = instance1.identify_datasets_info()
df_catalog = instance1.merge_additional_info()
instance1.save_to_csv('source_cnil_catalog')

# Uploading Files to GCS

In [ ]:
from classes.file_to_gcs import FromFileToGCS
import os

bucket_name = 'cnil_csv'
cred_path = 'cred/service_account_local_py.json'
init2 = FromFileToGCS(bucket_name, cred_path)
init2.create_bucket()
file_paths = ['data/catalog/source_cnil_catalog_2024-02-10.csv']
dest_folder = 'raw'
dest_blob = ['source_cnil_catalog_2024-02-15.csv']
init2.local_to_gcs(file_paths, dest_folder, dest_blob)

In [ ]:
bucket_name = 'cnil_csv'
cred_path = 'cred/service_account_local_py.json'
init2 = FromFileToGCS(bucket_name, cred_path)
init2.create_bucket()
url = ['https://www.data.gouv.fr/fr/organizations/cnil/datasets.csv', 'https://www.data.gouv.fr/fr/datasets/r/0f678674-4327-4c4d-8819-b6f508b41d0e']
dest_folder = 'raw'
dest_blob = ['datasets.csv', 'plaintes.csv']
init2.download_and_upload_from_URLs(url, dest_folder, dest_blob)

# Downloading from Catalog

In [ ]:
from classes.download_catalog_content import DlCatalogContent

instance3 = DlCatalogContent('data/catalog/source_cnil_catalog_2024-02-15.csv')
instance3.get_tables()
instance3.zip_files()

# Prep data to upload to BQ

In [ ]:
from classes.file_to_gcs import FromFileToGCS
import os

bucket_name = 'cnil_csv'
cred_path = 'cred/service_account_local_py.json'
init2 = FromFileToGCS(bucket_name, cred_path)
init2.create_bucket()
file_paths = ['data/raw_datasets.zip']
dest_folder = 'raw' 
init2.local_to_gcs(file_paths, dest_folder)

In [ ]:
from classes.prep_data import ZipFileProcessor

gcs_bucket_name = 'cnil_csv'
credential_path = 'cred/service_account_local_py.json'
zip_blob_name = '2024-02-15/raw/raw_datasets.zip'
output_folder_name = '2024-02-15/'+ 'prep'
instance4 = ZipFileProcessor(gcs_bucket_name, credential_path, zip_blob_name, output_folder_name)
zip_file = instance4.get_zip_file_object()

In [ ]:
import pandas as pd
from classes.prep_data import PrepDataCnilBQ

instance5 = PrepDataCnilBQ(zip_file)
zip_output = instance5.process_zip_file(zip_file)

In [ ]:
from classes.file_to_gcs import FromFileToGCS
bucket_name = 'cnil_csv'
cred_path = 'cred/service_account_local_py.json'
init2 = FromFileToGCS(bucket_name, cred_path)
init2.create_bucket()
file_paths = [zip_output]
dest_folder = 'prep'
init2.local_to_gcs(file_paths, dest_folder)

# GCS to GCP 

In [ ]:
from classes.prep_data import ZipFileProcessor

gcs_bucket_name = 'cnil_csv'
credential_path = 'cred/service_account_local_py.json'
zip_blob_name = '2024-02-17/prep/prep_datasets.zip'
output_folder_name = '2024-02-17/'+ 'prep'
instance4 = ZipFileProcessor(gcs_bucket_name, credential_path, zip_blob_name, output_folder_name)
zip_file = instance4.get_zip_file_object()

In [ ]:
from classes.gcs_to_gcp import FromGCStoGBQ

# usage exemple
credentials_path = 'cred/service_account_local_py.json'
project_id = 'cnil-392113'
dataset_name = 'raw_data'

processor_bq = FromGCStoGBQ(credentials_path, project_id, dataset_name)
processor_bq.create_dataset()
processor_bq.upload_zip_to_bq(zip_file)


# Building catalog from prep_data

In [ ]:
from classes.prep_data import ZipFileProcessor

gcs_bucket_name = 'cnil_csv'
credential_path = 'cred/service_account_local_py.json'
zip_blob_name = '2024-02-17/prep/prep_datasets.zip'
output_folder_name = '2024-02-17/'+ 'prep'
instance4 = ZipFileProcessor(gcs_bucket_name, credential_path, zip_blob_name, output_folder_name)
zip_file = instance4.get_zip_file_object()

In [ ]:
from classes.source_catalog import CustomCatalog
import io

instance8 = CustomCatalog('cred/service_account_local_py.json')
df = instance8.create_catalog_gcs(zip_file)
df

In [ ]:
import pandas as pd
csv_output = io.BytesIO()
df.to_csv(csv_output, index=False, sep=";")
csv_output.seek(0)

In [ ]:
from classes.file_to_gcs import FromFileToGCS

bucket_name = 'cnil_csv'
cred_path = 'cred/service_account_local_py.json'
init2 = FromFileToGCS(bucket_name, cred_path)
init2.create_bucket()
file_paths = [csv_output]
dest_folder = 'prep'
dest_blob = ['prepdata_cnil_catalog_2024-02-17.csv']
init2.local_to_gcs(file_paths, dest_folder, dest_blob)

# Building catalog from BQ raw_data

In [ ]:
from classes.source_catalog import CustomCatalog

credential_path = 'cred/service_account_local_py.json'
dataset_name = 'raw_data'
project_id = 'cnil-392113'
instance8 = CustomCatalog(credential_path, project_id, dataset_name)
df = instance8.bq_raw_catalog()

In [ ]:
from classes.gcs_to_gcp import FromGCStoGBQ

credentials_path = 'cred/service_account_local_py.json'
project_id = 'cnil-392113'
dataset_name = 'catalog_data'
table_name = 'cnil_catalog_bq'

processor_bq = FromGCStoGBQ(credentials_path, project_id, dataset_name)
processor_bq.create_dataset()
processor_bq.df_to_bq(df, table_name)